Calculating the mld depth using interpolation of the s-levels in the Norkyst dataset. This creates more continous depths and might make the mld calculations more accurate than using a transformation method from s_rho to z_rho. This notebook is to visualize the differences and compare these two methods of calculating the mld. I will use the xarray interpolation feature as recommended by a colleauge. Ill try using a linear method. If this interpolation lacks effiency, I might consider using xroms and sgrid instead, but I will try to stick with xarray.

In [1]:
from script_mld import dens
from script_mld import transformation
from script_mld import MLD
import xarray as xr
import matplotlib.pyplot as plt 
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import interp2d

In [16]:
#we try (30.03.2024)
#TODO sjekk at det blir riktig datasett å jobbe med - eller om de ønsker at jeg skal bruke operational
ds_2024 = xr.open_dataset(f'/lustre/storeB/project/fou/hi/foccus/datasets/symlinks/norkystv3-hindcast/2024/norkyst800-20240330.nc').isel(time = 0)
ds_2024.head() 

#first thing Victor does is to select out all variables and define variables + parameters and perform the vertical transformation
#Because I already have the transformation code, I will use that one

transformation(ds_2024)
z = ds_2024.z_rho

In [7]:
#hente ut temp også
temp = ds_2024.temperature
#og hente ut dimensjoner, så først sjekke datsettet og dimensjonene 
ds_2024

<xarray.Dataset> Size: 4GB
Dimensions:           (Y: 1148, X: 2747, s_rho: 40, s_w: 41)
Coordinates:
    time              datetime64[ns] 8B 2024-03-30
  * s_rho             (s_rho) float64 320B -0.9799 -0.9403 ... -0.004904
  * X                 (X) float64 22kB 0.0 800.0 1.6e+03 ... 2.196e+06 2.197e+06
  * Y                 (Y) float64 9kB 0.0 800.0 1.6e+03 ... 9.168e+05 9.176e+05
  * s_w               (s_w) float64 328B -1.0 -0.96 -0.9208 ... -0.01 0.0
    lon               (X, Y) float64 25MB 8.7 8.69 8.679 ... 18.38 18.35 18.33
    lat               (X, Y) float64 25MB 54.29 54.3 54.3 ... 75.72 75.72 75.73
    z_rho             (s_rho, X, Y) float64 1GB nan nan nan ... -0.8425 -0.843
Data variables: (12/18)
    Uwind_eastward    (Y, X) float32 13MB ...
    Vwind_northward   (Y, X) float32 13MB ...
    zeta              (Y, X) float32 13MB ...
    ubar_eastward     (Y, X) float32 13MB ...
    vbar_northward    (Y, X) float32 13MB ...
    u_eastward        (s_rho, Y, X) float32 505MB ...
    ...                ...
    hc                float64 8B ...
    Cs_r              (s_rho) float64 320B ...
    Cs_w              (s_w) float64 328B ...
    h                 (Y, X) float32 13MB ...
    projection_stere  int32 4B ...
    depth             int32 4B ...
Attributes: (12/33)
    id:                      15f95603-12d1-4e0f-8cbe-33946594447f
    naming_authority:        no.met
    operational_status:      scientific
    iso_topic_category:      oceans
    activity_type:           Numerical Simulation
    keywords_vocabulary:     GCMDSK:GCMD Science Keywords:https://gcmd.earthd...
    ...                      ...
    project:                 Norkyst_v3
    license:                 https://spdx.org/licenses/CC-BY-4.0 (CC-BY-4.0)
    title:                   Norkyst-800m - ROMS, Norkyst-800m ocean hindcast...
    summary:                 Norkyst-800m (Norwegian Coast 800m horizontal re...
    title_no:                Hindcast prognoser fra havmodellen Norkyst-800m,...
    summary_no:              NorKyst-800m (Norske kystområder med 800m horiso...

In [17]:
#dims
Nx = temp.shape[2]
Ny = temp.shape[1]
Nt = temp.shape[0]
Nz = ds_2024.s_w.shape[0]

#new depth to interpolate to 
new_depth = np.linspace(z.min().item(), z.max().item(), 200)
new_depth[-1] = z.max().item()

In [ ]:
#try Victors code
#just testing some things to make sure I understand the logistics of the code:

In [18]:
array = np.array([3,np.nan])

In [19]:
mask = np.isfinite(array)

In [23]:
print(mask)
new_depth = np.linspace(0, 2000, 200)

[ True False]


In [26]:
if mask.sum() < 2:
    test = np.full_like(new_depth, np.nan)
    print(f'The sum is less than two, the sum is: {mask.sum()}')
    print(f'New depth now consists of: {test}')
else:
    print(mask.sum())


The sum is less than two, the sum is: 1
New depth now consists of: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan]


In [27]:
#Okay so he interpolates as long as the existing depth array consists of minimum two finite points
#this is because two real points is necessary to be able to perform the interpolation